## Repliikide eraldamine ja andmebaasi loomine test5

In [1]:
from estnltk import Text
import re
import regex
from estnltk.taggers.system.rule_taggers import RegexTagger, Ruleset
from estnltk.taggers.system.rule_taggers import StaticExtractionRule
from estnltk.taggers import HeaderBasedSegmenter
from estnltk.taggers import NerTagger
import sqlite3
import os
import datetime
from estnltk.converters import text_to_json

Esiteks võetakse üks näidisfail:

In [2]:
test_file = open("test5/2019-09-07.txt", mode="r", encoding="utf-8")
test_text = test_file.read()
print(test_text[:300])

K01: Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?
-:
K02: Breivik tahtis proovida, mismoodi õhu lenn Sid raisakotkas kohe appi pahenduse, kui õhusegasse Aalto tegi, kaval linn. Natuurist haaras, pärdik öeldes kuuleb. Palun lenda sirgelt. Pal


In [3]:
print(re.findall(r'^.',test_text))
print(re.findall(r'((^|\n).)',test_text))
print(re.findall(r'((^|\n).*:)',test_text))

['K']
[('K', ''), ('\n-', '\n'), ('\nK', '\n'), ('\n-', '\n'), ('\nK', '\n'), ('\nA', '\n'), ('\nA', '\n'), ('\nK', '\n'), ('\nA', '\n'), ('\nK', '\n'), ('\nA', '\n'), ('\nA', '\n'), ('\nA', '\n'), ('\nK', '\n'), ('\nA', '\n'), ('\nA', '\n'), ('\nA', '\n'), ('\nK', '\n'), ('\nA', '\n'), ('\nK', '\n'), ('\nA', '\n'), ('\nA', '\n'), ('\nA', '\n'), ('\nA', '\n'), ('\nK', '\n'), ('\nA', '\n'), ('\nA', '\n'), ('\nA', '\n'), ('\nA', '\n'), ('\nK', '\n'), ('\nA', '\n'), ('\nI', '\n'), ('\nK', '\n'), ('\nA', '\n'), ('\nA', '\n'), ('\nA', '\n'), ('\nK', '\n'), ('\nA', '\n'), ('\nA', '\n'), ('\nT', '\n'), ('\nK', '\n'), ('\nA', '\n'), ('\nK', '\n'), ('\nA', '\n'), ('\nA', '\n'), ('\nN', '\n'), ('\nK', '\n'), ('\nA', '\n'), ('\nA', '\n'), ('\nK', '\n'), ('\nA', '\n'), ('\nI', '\n'), ('\nK', '\n'), ('\nA', '\n'), ('\nA', '\n'), ('\nK', '\n'), ('\nA', '\n'), ('\nA', '\n'), ('\nA', '\n'), ('\nK', '\n'), ('\nA', '\n'), ('\nA', '\n'), ('\n-', '\n')]
[('K01:', ''), ('\n-:', '\n'), ('\nK02:', '\n'), ('\

Kõnelejale vastav regulaaravaldis:

In [4]:
pattern = r'((^|\n)(?P<span>(?P<name>.*): ?))'

In [5]:
print([match.group('name') for match in re.finditer(pattern, test_text)])
print([match.group('span') for match in re.finditer(pattern, test_text)])
print([match.group() for match in re.finditer(pattern, test_text)])

['K01', '-', 'K02', '-', 'K06', 'Artur Talvik', 'Ainar Ruussaar', 'K07', 'Artur Talvik', 'K07', 'Artur Talvik', 'Ainar Ruussaar', 'Artur Talvik', 'K07', 'Artur Talvik', 'Ainar Ruussaar', 'Artur Talvik', 'Ainar Ruussaar', 'K07', 'Artur Talvik', 'Ainar Ruussaar', 'Artur Talvik', 'Ainar Ruussaar', 'K07', 'Artur Talvik', 'Ainar Ruussaar', 'Artur Talvik', 'Ainar Ruussaar', 'K07', 'Artur Talvik', 'Ignar Fjuk', 'K06', 'Artur Talvik', 'Ainar Ruussaar', 'Artur Talvik', 'K07', 'Ainar Ruussaar', 'Artur Talvik', 'K07', 'Artur Talvik', 'K07', 'Artur Talvik', 'Ainar Ruussaar', 'K07', 'Artur Talvik', 'Ainar Ruussaar', 'K07', 'Artur Talvik', 'Ignar Fjuk', 'K06', 'Artur Talvik', 'Ainar Ruussaar', 'K07', 'Ainar Ruussaar', 'Artur Talvik', 'Ainar Ruussaar', 'K07', 'Ainar Ruussaar', 'Artur Talvik', '-']
['K01: ', '-:', 'K02: ', '-:', 'K06: ', 'Artur Talvik: ', 'Ainar Ruussaar: ', 'K07: ', 'Artur Talvik: ', 'K07: ', 'Artur Talvik: ', 'Ainar Ruussaar: ', 'Artur Talvik: ', 'K07: ', 'Artur Talvik: ', 'Ainar Ru

Funktsioon kõneleja nime atribuudi lisamiseks:

In [6]:
def decorator(layer,span,annotation):
    annotation['person'] = annotation['match'].group(4)
    return annotation

Lisatakse reegel kõneleja fikseeriva tekstifragmendi väljavõtmiseks:

In [7]:
rule = [StaticExtractionRule(pattern=regex.Regex('((^|\n)(?P<span>(?P<name>.*): ?))'), group=3)]
ruleset = Ruleset()
ruleset.add_rules(rule)

Märgendaja RegexTagger loomine:

In [8]:
tagger = RegexTagger(
    ruleset = ruleset,
    output_layer = 'headers',
    output_attributes = ['person'],
    decorator=decorator,
    match_attribute='match'
) 

Katsetus näidisteksti peal:

In [9]:
tagged_test_text = tagger.tag(Text(test_text))
display(tagged_test_text.headers[:5])

Layer(name='headers', attributes=('person',), spans=SL[Span('K01: ', [{'person': 'K01'}]),
Span('-:', [{'person': '-'}]),
Span('K02: ', [{'person': 'K02'}]),
Span('-:', [{'person': '-'}]),
Span('K06: ', [{'person': 'K06'}])])

Märgendaja HeaderBasedSegmenter loomine:

In [10]:
tagger2 = HeaderBasedSegmenter(
    input_layer = 'headers', 
    output_layer = 'lines',
    output_attributes = ['person'],
    decorator = lambda header_span: {'person': header_span['person'][0]},
    include_header = False
)

In [11]:
tagger2.tag(tagged_test_text)
tagged_test_text.lines[:5]

Layer(name='lines', attributes=('person',), spans=SL[Span('Kuku raadios välja öeldud seisukohad ei pea ühtima Kuku raadio seisukohtadega. Te kuulate Kuku raadiot?\n', [{'person': 'K01'}]),
Span('\n', [{'person': '-'}]),
Span('Breivik tahtis proovida, mismoodi õhu lenn Sid raisakotkas kohe appi pahenduse, kui õhusegasse Aalto tegi, kaval linn. Natuurist haaras, pärdik öeldes kuuleb. Palun lenda sirgelt. Palun lenda kõrge. Võtta asja kerge, tappa maha, ära päädhabi.\n', [{'person': 'K02'}]),
Span('\n', [{'person': '-'}]),
Span('Ta oli liiga uudis ainult koduvabariigist.\n', [{'person': 'K06'}])])

Nimeüksuste märgendaja NerTagger:

In [12]:
nertagger = NerTagger()

Luuakse andmebaas ja seal uus tabel, kuhu lisatakse kõik repliikidele vastavad Text-objektid, samuti info kuupäeva, kõneleja nime ja repliigi numbri kohta.

In [18]:
con = sqlite3.connect("media_data.db")
cur = con.cursor()
cur.execute('pragma encoding=UTF8')
cur.execute("CREATE TABLE lines(ID INTEGER PRIMARY KEY, line TEXT, number INTEGER, person TEXT, date DATE)")

In [19]:
path = 'test5/'
for file in os.listdir(path):
    with open(os.path.join(path, file), mode="r", encoding="utf-8") as f:
        raw_text = f.read()
        filename = file.replace('.txt', '')
        date = datetime.datetime.strptime(filename, "%Y-%m-%d").date()
        
        text = tagger.tag(Text(raw_text))
        text = tagger2.tag(text)
        
        for i in range(len(text.lines)):
            span = text.lines[i]
            line = span.text.strip()
            if line == '':
                continue
            number = i+1
            line = Text(line)
            line.meta['line_number'] = number
            line.meta['person'] = span.person
            line.meta['date'] = filename
            line.tag_layer(['sentences', 'morph_analysis'])
            nertagger.tag(line)
            line_json = text_to_json(line)
            cur.execute("""INSERT INTO lines
                                  (line, number, person, date) 
                                  VALUES (?, ?, ?, ?);""", (line_json, number, span.person, date))
            con.commit()

In [20]:
con.close()